# iFLYTEK 长光卫星——[高分辨率遥感影像耕地地块提取挑战赛](http://challenge.xfyun.cn/topic/info?type=plot-extraction-2021)（非官方baseline）
**<font color=red>注意：本项目不提供赛题数据集，若训练，可通过上方链接从官网报名下载数据集</font>**

## 一、赛事背景
耕地的数量和质量是保证我国农业可持续发展的关键。遥感技术可以准确、及时地探测地表信息，客观地反映地物的状态及其变化。利用高分辨率遥感影像准确地提取耕地地块是精准农业的基本任务。目前提取高精度的耕地地块产品主要还是依靠人工解译的手段，费时费力。传统耕地地块提取技术的效率已无法满足当前精细化农业应用快速响应的需求。高分辨率遥感影像耕地地块的自动化提取算法依然是国内外遥感领域共同面临的科学问题。因此，本赛道旨在充分利用遥感大数据、人工智能等先进技术，实现高效、可用、实用的先进算法，以提升自动化提取高分辨率遥感影像中耕地地块的能力。

## 二、赛事任务
我国幅员辽阔，由于自然地理环境差异较大，耕地类型因地而异，精细农业需要强大的数据作为支撑，本次大赛以吉林一号高分辨率卫星遥感影像作为数据集，参赛选手需基于提供的样本构建模型，预测未知影像中耕地的矢量边界范围。

## 三、数据说明
本次大赛提供吉林一号高分辨率遥感影像作为数据源，影像为四通道数据（B,G,R,NIR），分辨率为0.75~1.1米之间。包括训练数据与测试数据，由长光卫星技术有限公司拍摄、标注、构建。其中，初赛数据集包括16张不同尺寸的tif图片。

a. 原始影像

影像格式为tif，包含B、G、R、NIR四个波段，训练集影像为一系列尺寸不固定的遥感影像（行数与列数≥3000），用于初赛与复赛的测试集影像尺寸可变。

<img src="https://ai-studio-static-online.cdn.bcebos.com/a4c3288b3e9b4d2f8fcde2e89d60315de6d9fa830d32460492dee3b39cec83d8" width=400/>


b.	标签数据

标签格式为矢量数据（shapefile），标签数据的名称与原始影像一致。标签数据中的特征类型为多边形即耕地地块范围以多边形方式进行勾画

出于数据安全保证的考虑，所有数据均为脱敏处理后的数据，地理坐标加偏移。 此次比赛分为初赛和复赛两个阶段，两个阶段的区别是所提供的测试数据不同，其他的设置均相同。

<img src="https://ai-studio-static-online.cdn.bcebos.com/6255349b2d634013b1cfcc19e1d9dff20d3d40ea2fd54a9796962faf0e0e4d5d" width=1000/>

## 四、比赛思路

### 1.文件格式转化
该比赛任务明显为语义分割，需要将地图数据划分为`0：背景`以及`1：耕地`这两种类别，可以使用`PaddleSeg`来完成。首先将tif图转化为png图以适配`PaddleSeg`，并需要将shp文件转化为png或jpg格式的图片标注文件。

<font color=red>注：由于本作者还未找到在AI Studio中安装GDAL库的方案，所以该代码只能在装有python第三方库GDAL的本地运行。</font>

tif转png示例代码：

```
from PIL import  Image
Image.MAX_IMAGE_PIXELS = None
im = Image.open('JPEGImages/CGSH_1_offset.tif').convert('RGBA')
im.save('JPEGImages/CGSH_1_offset.png')
```

shp文件转化png示例代码：

```
import os
import re
import fnmatch
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import geopandas as gpd
from osgeo import gdal
import matplotlib.pyplot as plt
Image.MAX_IMAGE_PIXELS = None

ROOT = r'F:/ChangguangSatellite_Coffe'
img_path = 'JPEGImages'
shp_path = 'Annotations'
# root path for saving the mask.
ROOT_DIR = ROOT + '/train'
IMAGE_DIR = ROOT_DIR+"/"+img_path
ANNOTATION_DIR = ROOT_DIR+'/'+shp_path


def filter_for_annotations(root, files):
    file_types = ['*.shp']
    file_types = r'|'.join([fnmatch.translate(x) for x in file_types])
    files = [os.path.join(root, f) for f in files]
    files = [f for f in files if re.match(file_types, f)]
    return files

if __name__ == "__main__":
    with open(os.path.join(ROOT_DIR,'train.txt'),'w',encoding='utf-8') as fp:
        for root, _, files in os.walk(ANNOTATION_DIR):
            annotation_files = filter_for_annotations(ANNOTATION_DIR, files)
            for segpath in annotation_files:
                result = gdal.Warp('masked.png'
                   , os.path.join(IMAGE_DIR, os.path.splitext(os.path.basename(segpath))[0])+'.tif'
                   , cutlineDSName=os.path.join(ANNOTATION_DIR, os.path.splitext(os.path.basename(segpath))[0])+'.shp')
                arr=result.ReadAsArray()[0]
                arr[arr>0]=1 #这里将数组中大于0的数据全部标为1
                im = Image.fromarray(arr).convert('L')
                im.save(os.path.join(ANNOTATION_DIR, os.path.splitext(os.path.basename(segpath))[0])+'.png')
                del result
                fp.write(img_path+'/'+os.path.splitext(os.path.basename(segpath))[0]+'.png '+
                         shp_path+'/'+os.path.splitext(os.path.basename(segpath))[0]+'.png\n')
                print(ANNOTATION_DIR+'/'+os.path.splitext(os.path.basename(segpath))[0]+'.png')
```

将生成的图片文件以及标注文件打包上传到AI Studio上，解压缩并将文件命名如下：

- labels.txt	(标签类别，由于是二分类，所以只需要输入如：plough一行即可)
- train.txt		（训练集，每行都包含训练的图片路径以及对应的标注路径，并使用空格进行分割）
- val.txt		（验证集，每行都包含验证的图片路径以及对应的标注路径，并使用空格进行分割）
- test.txt		（测试集，每行都包含测试的图片路径）

### 2.数据集增强

这里手动进行了一下数据集的增强，使用`PIL.ImageEnhance`对图片进行色度、对比度、亮度以及清晰度的扩增。使用扩增后的数据集进行训练，准确率提升约3个百分点。

In [ ]:
import os
from PIL import Image,ImageEnhance
Image.MAX_IMAGE_PIXELS = None

color_aug=[0.1,0.2,0.3]
bright_aug=[0.5,2.0]
contrast_aug=[0.5,2.0]
sharp_aug=[0.0,0.5,2.0,4.0]

def enhance(line,fr,fw):
    fw.write(line)
    img_path = line.strip().split(' ')[0]
    ann_path = line.strip().split(' ')[1]
    img = Image.open(img_path)
    for i,rate in enumerate(color_aug):
        aug_img = ImageEnhance.Color(img).enhance(rate)
        aug_img.save(img_path.split('.')[0]+'_c{}.png'.format(i+1))
        fw.write(img_path.split('.')[0]+'_c{}.png'.format(i+1)+ ' ' + ann_path+'\n')
    for i,rate in enumerate(bright_aug):
        aug_img = ImageEnhance.Brightness(img).enhance(rate)
        aug_img.save(img_path.split('.')[0]+'_b{}.png'.format(i+1))
        fw.write(img_path.split('.')[0]+'_b{}.png'.format(i+1)+ ' ' + ann_path+'\n')
    for i,rate in enumerate(contrast_aug):
        aug_img = ImageEnhance.Contrast(img).enhance(rate)
        aug_img.save(img_path.split('.')[0]+'_ct{}.png'.format(i+1))
        fw.write(img_path.split('.')[0]+'_ct{}.png'.format(i+1)+ ' ' + ann_path+'\n')
    for i,rate in enumerate(sharp_aug):
        aug_img = ImageEnhance.Sharpness(img).enhance(rate)
        aug_img.save(img_path.split('.')[0]+'_s{}.png'.format(i+1))
        fw.write(img_path.split('.')[0]+'_s{}.png'.format(i+1)+ ' ' + ann_path+'\n')

with open('train.txt','r',encoding='utf-8') as fr:
    with open('enhance_train.txt','w',encoding='utf-8') as fw:
        lines = fr.readlines()
        for line in lines:
            enhance(line,fr,fw)

with open('val.txt','r',encoding='utf-8') as fr:
    with open('enhance_val.txt','w',encoding='utf-8') as fw:
        lines = fr.readlines()
        for line in lines:
            enhance(line,fr,fw)

## 五、使用PaddleSeg进行模型训练

PaddleSeg详细使用指南见：[PaddleSeg官方文档](https://paddleseg.readthedocs.io/zh_CN/release-2.1)

In [ ]:
#下载所需的包
!pip install scikit-image
!pip install geopandas
!pip install shapely

### 1.使用ocrnet_hrnetw48_voc12aug_512x512_40k.yml进行训练

**<font size=3>（1）ocrnet</font>**

Paper：[Segmentation Transformer: Object-Contextual
Representations for Semantic Segmentation](https://arxiv.org/pdf/1909.11065.pdf)

![](https://ai-studio-static-online.cdn.bcebos.com/18f4a49387604fc9b936bf820b84a591d264c71853f245c8b99a7277630bd210)

<font size=3>ocrnet提出了使用Transformer编码器-解码器框架重新表述对象上下文的方案。
  
  在解码器中，前两步即目标区域学习和目标区域表示计算被集成为交叉注意模块：用于对像素进行分类的线性投影（即生成目标区域）是类别查询，目标区域表示是交叉注意输出。
  
  最后一步是添加到编码器的交叉注意模块，其中键和值是解码器输出，查询是每个位置的表示。</font>

<img src="https://ai-studio-static-online.cdn.bcebos.com/be251f34521d4ea4b1b7a48dc62a31c4779b33c2c0804d2cb16d087937c9fd70" width=1000/>




**<font size=3>（2）对配置文件进行修改</font>**

配置文件说明可见：[配置项](https://paddleseg.readthedocs.io/zh_CN/release-2.1/design/use/use.html)
  
<font color=red>注意：这里要使用RandomPaddingCrop对图片进行训练时增强</font>
|ocrnet_hrnetw18_voc12aug_512x512_40k.yml|cityscapes.yml|
|---|---|
|![ocrnet_hrnetw18_voc12aug_512x512_40k.yml](https://ai-studio-static-online.cdn.bcebos.com/b90708b990e6461a8be6955410babc1fab071ec41b3948e28a452e299d569f10)|![cityscapes.yml](https://ai-studio-static-online.cdn.bcebos.com/65cf11c6416240809cc9abe661f4ebdfd5765c7be9d84d9d9382decd435bfc82)|

（3）模型训练

模型训练配置见：[模型训练](https://paddleseg.readthedocs.io/zh_CN/release-2.1/train/train.html)

<font color=red>注意：方案中在进行图片验证时，首先将图片动态切割为多个较小的图片，然后一一进行预测，最终将预测图拼接成一张总的预测图片，修改代码见`PaddleSeg/paddleseg/core/val.py`</font>

In [ ]:
!python PaddleSeg/train.py --config PaddleSeg/configs/ocrnet/ocrnet_hrnetw48_voc12aug_512x512_40k.yml \
        --iters 19200 \
        --learning_rate 0.0125 \
        --save_interval 192 \
        --do_eval \
        #--resume_model 'output/ocr_best_14592'

In [2]:
#删除多余的训练模型
!rm -rf ./output/iter*

<font size=3>这里除ocrnet外，还提供了其他几个模型，有兴趣可以将ocrnet替换为下列模型</font>
> deeplabv3/deeplabv3_resnet50_os8_voc12aug_512x512_40k.yml

> deeplabv3/deeplabv3_resnet101_os8_voc12aug_512x512_40k.yml

> deeplabv3p/deeplabv3p_resnet50_os8_voc12aug_512x512_40k.yml

> deeplabv3p/deeplabv3p_resnet101_os8_voc12aug_512x512_40k.yml

> ocrnet/ocrnet_hrnetw48_voc12aug_512x512_40k.yml

> ocrnet/ocrnet_hrnetw18_voc12aug_512x512_40k.yml

> fcn/fcn_hrnetw18_voc12aug_512x512_40k.yml

> fcn/fcn_hrnetw48_voc12aug_512x512_40k.yml

## 六、模型预测

<font color=red>**注意：**
  
  1.由于本作者还未找到在AI Studio中安装GDAL库的方案，所以预测代码只能在装有python第三方库GDAL的本地运行。</font>

  <font color=red>2.方案中在进行图片预测时，同样首先将图片动态切割为多个较小的图片，然后一一进行预测，最终将预测图拼接成一张总的预测图片，修改代码见`PaddleSeg\paddleseg\core\predict.py`。</font>此外，该文件中设置了形态学中的开运算。由于获取每个预测多边形时，可能会存在将预测多边形包含于其他多边形内部的情况，针对这种情况代码进行了两种处理方式：**将包含于内部的小多边形视为孔洞进行去除**以及**忽视包含于内部的小多边形**，通过设置参数`cut_hole`来进行选择。
  
3.shp文件生成代码见`PaddleSeg\Shp_data_out.py`

In [ ]:
%cd PaddleSeg
#该代码同PaddleSeg/test.py
import os
import paddleseg
from paddleseg.models import U2Net,DeepLabV3,FCN,OCRNet
from paddleseg.models.backbones.resnet_vd import ResNet50_vd
from paddleseg.models.backbones.hrnet import HRNet_W18,HRNet_W48

Root = '/home/aistudio'
image_dir=os.path.join(Root,'test')

#backbone= ResNet50_vd()
backbone= HRNet_W48()

model = OCRNet(
    backbone=backbone,
    backbone_indices=[0],
    num_classes=2,
)

model_path = os.path.join(Root,'output/ocr_best_10752/model.pdparams') #最优模型路径

# 构建验证用的transforms
import paddleseg.transforms.transforms as T
transforms = T.Compose([
    T.Normalize()
])

with open(os.path.join(image_dir,'test.txt'),'r',encoding='utf-8') as fp:
    lines=fp.readlines()
    image_list=[os.path.join(image_dir,line.strip()) for line in lines]

paddleseg.core.predict(
                    model,
                    model_path,
                    transforms,
                    image_list = image_list,
                    image_dir=image_dir,
                    save_dir=os.path.join(Root, 'output/predict'),
                    aug_pred=False,
                    scales=1.0,
                    flip_horizontal=False,
                    flip_vertical=False,
                    is_slide=False,
                    stride=None,
                    shape=(512,512),
                    kernel_size=(10,10),#开运算核
                    MORPH_OPEN=True,#是否执行开运算
                    MORPH_OPEN_NUM = 3,#执行开运算次数
                    cut_hole = True,#是否去掉预测多边形中的孔洞
                    crop_size=None
                    )

## 七、效果展示



| 原图 | 预测图 | shp文件可视化 |
| -------- | -------- | -------- |
| ![](https://ai-studio-static-online.cdn.bcebos.com/fba4dd5765fe410cb928a43e239005b7b7047e8518ee4556a4a0ccfe90f598ee) | ![](https://ai-studio-static-online.cdn.bcebos.com/088de4ea73d841c696239faece4b86b3882494b75acc438da8ac048fd2709a08) | ![](https://ai-studio-static-online.cdn.bcebos.com/3ff785fc7ddc4a6d940f2c5a8e6f128f433822f21c0e4bd680a0f85648a99bc5)  |






## 八、总结

比赛图片实在太大，即使是32G显存的AI Studio训练也还是十分吃紧，此外，训练结果不算太理想，miou在32上，与前排大佬还是有很大差距，希望有大佬能够点出不足。

>本人武汉科技大学信息安全专业毕业，今年考入本校研究生，在CV和NLP上都有兴趣~

>我在AI Studio上获得钻石等级，点亮8个徽章

>欢迎关注[我的主页](https://aistudio.baidu.com/aistudio/personalcenter/thirdview/71231)

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 